In [13]:
import pandas as pd
import numpy as np
df = pd.read_csv("../data/misgrosi.csv")

In [18]:
df.head()

,Name,Intro,Location,Role
0,Nani Moni,Einfach ein lässiges Grosi sy,4900 Langenthal,Tages-Grosi
1,Jida,Betreue Kinder aus allen Kulturen,4051 Basel,Tages-Grosi
2,Phoebe-Grosi,Bin da offen und neugierig,8105 Watt,Tages-Grosi
3,Wunsch-Omi Elvira,"Jungebliebene, aktive mitten im Leben stehende...",9533 Kirchberg,Tages-Grosi
4,Grosi claudia,Bin aufgestellt und kinder lieb,3363 Oberön,Tages-Grosi


In [26]:
def post(row):
    try:
        post_stadt = row['Location'].split()
        if post_stadt[0].isdigit():
            val = post_stadt[0]
        else:
            val = None
    except:
        val = str(row['Location'])
    
    return val
        
df['Post Code'] = df.apply(post, axis=1)

0      4900
1      4051
2      8105
3      9533
4      3363
      ...  
95     None
96     4051
97     4416
98    80992
99     8064
Name: Post Code, Length: 100, dtype: object

In [30]:
def stadt(row):
    try:
        post_stadt = row['Location'].split()

        if len(post_stadt) == 2 :
            val = post_stadt[1]
        elif not post_stadt[0].isdigit():
            val = post_stadt[0]
        else:
            val = None
    except:
        val = None
    return val
        
df['Stadt'] = df.apply(stadt, axis=1)

In [31]:
df.to_csv("../data/misgrosi_candidate.csv", index=False) 

In [32]:
role_dict = {}
for i in df['Role']:
    if i in role_dict:
        role_dict[i] += 1
    else:
        role_dict[i] = 1
role_dict

{'Tages-Grosi': 27,
 'Kinderbetreuung allgemein': 43,
 'Grosi als Nanny / Haushaltshilfe': 11,
 'Babysitting': 9,
 'Leihoma für das Ausland': 1,
 'Babysitting, Express-Kinderbetreuung, Grosi als Nanny / Haushaltshilfe, Kinderbetreuung allgemein, Oma & Opa Kinderbetreuung, Tages-Grosi': 1,
 'Oma & Opa Kinderbetreuung': 4,
 'Grosi als Au-pair': 3,
 'Babysitting, Tages-Grosi': 1}